In [4]:
import cv2
import numpy as np

from pathlib import Path
import os
import sys
import onnx
import onnxruntime
from collections import defaultdict
# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# from src.config import MODEL_PATH, INPUT_SHAPE,OUTPUT_NAME,INPUT_NAME,session,CLASS_NAMES


In [5]:
# Define the path to your model
MODEL_PATH = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'
SEVERITY_THRESHOLD=10
# Function to load the model and create a session
def load_model(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)
    # Create an ONNX Runtime session
    session = onnxruntime.InferenceSession(model_path)
    return model, session

# Load the model and session
model, session = load_model(MODEL_PATH)

# Get model input and output details
INPUT_NAME = session.get_inputs()[0].name
INPUT_SHAPE = session.get_inputs()[0].shape
OUTPUT_NAME = session.get_outputs()[0].name

# Optionally, load class names from the model metadata
CLASS_NAMES = {}
for prop in model.metadata_props:
    if prop.key == "names" or prop.key == "classes":
        CLASS_NAMES=eval(prop.value)  # Be cautious with eval
        break

In [6]:
OUTPUT_NAME

'output0'

In [52]:
print (INPUT_NAME)
print (INPUT_SHAPE)
print (OUTPUT_NAME)
print(CLASS_NAMES)
dets=""

images
[1, 3, 640, 640]
output0
{0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}


In [1]:
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
def load_model(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    return session

# Preprocess the image for YOLOv5 (resize, normalize, and convert to tensor format)
def preprocess_image(image_path, input_size=(640, 640)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0  # Normalize to [0, 1]
    img_transposed = np.transpose(img_normalized, (2, 0, 1))  # HWC to CHW format
    img_tensor = np.expand_dims(img_transposed, axis=0).astype(np.float32)  # Add batch dimension
    return img_tensor, img

# Post-process the model output to extract bounding boxes, class labels, and probabilities
def postprocess_output(output, img_shape, conf_threshold=0.5, iou_threshold=0.4):
    boxes, scores, class_ids = [], [], []
    detections = output[0]
    img_h, img_w = img_shape[:2]

    # Parse each detection (x, y, w, h, conf, class_score1, class_score2, ..., class_scoreN)
    for detection in detections:
        scores = detection[4:]  # All the class scores
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            # Box is (center_x, center_y, width, height) - convert to x1, y1, x2, y2
            box = detection[:4] * np.array([img_w, img_h, img_w, img_h])
            x_center, y_center, width, height = box
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            boxes.append([x1, y1, x2, y2])
            scores.append(confidence)
            class_ids.append(class_id)
    
    return boxes, scores, class_ids

# Map class IDs to class names
class_names = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

# Inference function
def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Post-process the output
    boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    # Number of detected parasites
    num_parasites = len(class_ids)
    
    # List each detected parasite with label and confidence
    results = []
    for i in range(num_parasites):
        label = class_names[class_ids[i]]
        confidence = confidences[i]
        results.append((label, confidence))

    return num_parasites, results

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_path = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/PO/po_52.jpg'  # Update with your image path

num_parasites, detection_results = run_inference(onnx_model_path, image_path)

print(f"Number of detected parasites: {num_parasites}")
for label, confidence in detection_results:
    print(f"Detected {label} with confidence {confidence:.2f}")


IndexError: index 119950 is out of bounds for axis 0 with size 25196

In [76]:
class TempStorage:
    def __init__(self):
        self.patient_detections = defaultdict(list)

    def save_detection(self, patient_id, image_id, detection_results):
        self.patient_detections[patient_id].append({
            "image_id": image_id,
            "results": detection_results
        })

    def get_patient_detections(self, patient_id):
        return self.patient_detections.get(patient_id, [])

# def aggregate_and_assess_severity(image_results, severity_threshold):
#     aggregated_results = {}

#     # Aggregate data across all images
#     for result in image_results:
#         for parasite in result["results"]:
#             parasite_type = parasite["class_name"]
#             confidence = parasite["confidence"]

#             if parasite_type not in aggregated_results:
#                 aggregated_results[parasite_type] = {
#                     "count": 0,
#                     "total_confidence": 0.0,
#                     "instances": 0
#                 }

#             aggregated_results[parasite_type]["count"] += 1
#             aggregated_results[parasite_type]["total_confidence"] += confidence
#             aggregated_results[parasite_type]["instances"] += 1

#     # Calculate average confidence
#     for parasite_type, data in aggregated_results.items():
#         data["average_confidence"] = (
#             data["total_confidence"] / data["instances"]
#             if data["instances"] > 0 else 0
#         )

#     # Determine severity
#     total_parasite_count = sum(data["count"] for data in aggregated_results.values())
#     severity = "Mild" if total_parasite_count < severity_threshold else "Severe"

#     return aggregated_results, severity


def aggregate_and_assess_severity(image_results, severity_threshold):
    """
    Aggregate parasite detection results and assess severity based on the WBC count.

    Parameters:
    - image_results: List of dictionaries containing detection results for each image.
    - severity_threshold: The threshold ratio for determining severity.

    Returns:
    - Aggregated results for each parasite type, including average confidence.
    - Severity assessment based on adjusted parasite-to-WBC ratio.
    """
    aggregated_results = {}
    total_parasite_count = 0
    total_wbc_count = 0

    # Iterate through each image result
    for result in image_results:
        for parasite in result["results"]:
            parasite_type = parasite["class_name"]
            confidence = parasite["confidence"]

            if parasite_type.lower() == "wbc":
                # Count WBCs
                total_wbc_count += 1
            else:
                # Aggregate data for other parasites
                if parasite_type not in aggregated_results:
                    aggregated_results[parasite_type] = {
                        "count": 0,
                        "total_confidence": 0.0,
                        "instances": 0
                    }

                aggregated_results[parasite_type]["count"] += 1
                aggregated_results[parasite_type]["total_confidence"] += confidence
                aggregated_results[parasite_type]["instances"] += 1

        # Increment total parasite count (excluding WBCs)
        total_parasite_count += len([p for p in result["results"] if p["class_name"].lower() != "wbc"])

    # Calculate average confidence for each parasite type
    for parasite_type, data in aggregated_results.items():
        data["average_confidence"] = (
            data["total_confidence"] / data["instances"]
            if data["instances"] > 0 else 0
        )
        # Remove no longer needed fields
        del data["total_confidence"]
        del data["instances"]

    # Assess severity based on the adjusted ratio of parasites to WBCs
    if total_wbc_count > 0:
        # Adjusting the parasite density
        adjusted_parasite_density = (total_parasite_count * 8000) / total_wbc_count
    else:
        adjusted_parasite_density = 0
    
    severity = "Mild" if adjusted_parasite_density < severity_threshold else "Severe"

    # Output for debugging
    print(f"Total WBC Count: {total_wbc_count}")
    print(f"Adjusted Parasite Density: {adjusted_parasite_density}")

    return aggregated_results, severity




In [78]:
# Hardcoded patient ID and image folder path
patient_id = "pat_001"
image_folder = "/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/notebook/data/test_data"  # Ensure the case matches

# Log the received request and paths
print(f"Processing images for patient ID: {patient_id} in folder: {image_folder}")

# Create a temporary storage instance
temp_storage = TempStorage()

# Process the images for the given patient
process_patient_images(patient_id, image_folder, temp_storage)

# Aggregate results and assess severity
aggregated_results, severity = aggregate_and_assess_severity(
    temp_storage.get_patient_detections(patient_id), 0.1
)

# Log the results
# print(f"Aggregated Instances: {aggregated_results['instances']}")
print(f"Aggregated Instances: {aggregated_results}")

print(f"Severity: {severity}")


Processing images for patient ID: pat_001 in folder: /Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/notebook/data/test_data
Total WBC Count: 105
Adjusted Parasite Density: 23542.85714285714
Aggregated Instances: {'pf': {'count': 309, 'average_confidence': 0.9923047035643198}}
Severity: Severe


In [56]:
temp_storage.get_patient_detections(patient_id)

[{'image_id': 'pat_001_po_54',
  'results': [{'class': 2,
    'class_name': 'po',
    'confidence': 0.9979743,
    'bbox': [480.3688659667969,
     297.6835021972656,
     29.740924835205078,
     43.55450439453125]},
   {'class': 2,
    'class_name': 'po',
    'confidence': 0.77812135,
    'bbox': [600.99560546875,
     302.5611267089844,
     23.91472625732422,
     28.816410064697266]},
   {'class': 2,
    'class_name': 'po',
    'confidence': 0.7589141,
    'bbox': [600.998046875,
     302.91888427734375,
     23.721464157104492,
     29.460336685180664]},
   {'class': 4,
    'class_name': 'wbc',
    'confidence': 0.93385327,
    'bbox': [600.9099731445312,
     303.4964904785156,
     23.655609130859375,
     29.925935745239258]},
   {'class': 4,
    'class_name': 'wbc',
    'confidence': 0.84561646,
    'bbox': [601.3798217773438,
     371.83282470703125,
     25.746761322021484,
     36.308982849121094]},
   {'class': 4,
    'class_name': 'wbc',
    'confidence': 0.8624445,
    

In [93]:
def aggregate_and_assess_severity(image_results, severity_threshold):
    aggregated_results = {}

    # Aggregate data across all images
    for result in image_results:
        seen_parasites = set()  # To avoid double counting in the same image
        for parasite in result["results"]:
            parasite_type = parasite["class_name"]
            confidence = parasite["confidence"]

            # Unique key to identify each detected instance by class and bounding box
            unique_key = (parasite_type, tuple(parasite["bbox"]))

            if unique_key not in seen_parasites:
                seen_parasites.add(unique_key)

                if parasite_type not in aggregated_results:
                    aggregated_results[parasite_type] = {
                        "count": 0,
                        "total_confidence": 0.0,
                        "instances": 0
                    }

                aggregated_results[parasite_type]["count"] += 1
                aggregated_results[parasite_type]["total_confidence"] += confidence
                aggregated_results[parasite_type]["instances"] += 1

    # Calculate average confidence and severity, and print results
    severity_results = {}
    for parasite_type, data in aggregated_results.items():
        data["average_confidence"] = (
            data["total_confidence"] / data["instances"]
            if data["instances"] > 0 else 0
        )
        # Example severity computation: (count * average_confidence)
        severity = data["count"] * data["average_confidence"]

        # Assess severity based on the threshold
        severity_level = "low"
        if severity > severity_threshold:
            severity_level = "high"
        
        # Store the results
        severity_results[parasite_type] = {
            "severity": severity,
            "severity_level": severity_level,
            "count": data["count"],
            "average_confidence": data["average_confidence"]
        }
        
        # Print the aggregated results and severity level
        print(f"Parasite Type: {parasite_type}")
        print(f"  Count: {data['count']}")
        print(f"  Average Confidence: {data['average_confidence']:.2f}")
        print(f"  Severity: {severity:.2f}")
        print(f"  Severity Level: {severity_level}")
        print()

    return severity_results

In [45]:
aggregate_and_assess_severity(temp_storage.get_patient_detections(patient_id), 2)

({}, 'Mild')

In [22]:


def process_patient_images(patient_id, image_folder, temp_storage):
    # Ensure the image folder path is a Path object
    image_folder = Path(image_folder)
    
    # Get all image files in the folder
    image_files = [f for f in image_folder.iterdir() if f.is_file() and f.suffix.lower() in ('.jpg', '.jpeg', '.png', '.bmp')]
    
    for image_file in image_files:
        # Generate an image ID based on the filename
        image_id = f"{patient_id}_{image_file.stem}"
        
        # Preprocess the image
        image = preprocess_image(str(image_file), INPUT_SHAPE)
        
        # Run inference
        detections = run_inference(image,OUTPUT_NAME,INPUT_NAME)
        # print("God ypur help please")
        print(detections[0][5])

In [23]:
# Create a temporary storage instance
temp_storage = TempStorage()
# Process the images for the given patient
process_patient_images("pat_001", image_folder, temp_storage)

[4.7148270e+01 5.2467499e+00 1.2615450e+01 1.1524083e+01 4.5996565e-01
 9.9510700e-01 3.0811727e-03 1.2518764e-03 5.0483942e-03 1.4372170e-03]
[4.2691875e+01 6.7036638e+00 1.3287254e+01 1.0462936e+01 3.8295984e-05
 8.5345000e-01 2.7611732e-02 2.0596385e-02 5.3753108e-02 4.1178793e-02]
[4.7227016e+01 2.6920018e+00 1.3128720e+01 8.4899340e+00 6.7353249e-06
 8.7337136e-01 3.2094687e-02 2.5029212e-02 3.4123927e-02 3.3446729e-02]
[4.6573906e+01 8.0937805e+00 1.3256131e+01 1.3263679e+01 6.7737699e-04
 6.5975505e-01 3.6992580e-02 1.5244573e-02 3.3998609e-02 1.1052409e-01]
[4.1696060e+01 6.7956905e+00 1.6050461e+01 1.0169741e+01 4.4524670e-05
 4.3157393e-01 1.5692443e-02 4.0283409e-01 2.9539376e-02 2.7504712e-02]
[4.2823410e+01 7.3916988e+00 1.6442541e+01 1.0011182e+01 1.9460917e-05
 4.3370885e-01 1.1745256e-01 1.6490281e-02 2.4981886e-02 1.4584905e-01]
[4.3478996e+01 2.0824399e+00 1.2059678e+01 8.5652437e+00 3.0696392e-06
 8.6664701e-01 3.6396891e-02 4.1527808e-02 2.6181310e-02 3.7993312e-02]

In [29]:
import onnxruntime as ort

ort_session = ort.InferenceSession(MODEL_PATH)

# Prepare input
input_name = ort_session.get_inputs()[0].name
img = cv2.imread("po_52.jpg")
# Resize the image to the expected size (640x640)
img = cv2.resize(img, (640, 640))

# Convert to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Normalize pixel values to [0, 1]
img = img.astype(np.float32) / 255.0

# Transpose the dimensions to (channels, height, width)
img = np.transpose(img, (2, 0, 1))

# Add batch dimension
img = np.expand_dims(img, axis=0)

print(img.shape)  # Should print (1, 3, 640, 640)

# Prepare input for ONNX Runtime
ort_inputs = {ort_session.get_inputs()[0].name: img}

try:
    # Run inference
    ort_outs = ort_session.run(None, ort_inputs)
    print(ort_outs)
    # Process the output (you'll need to add this part based on your model's output)
    # ...
except Exception as e:
    print(f"Error occurred: {str(e)}")

(1, 3, 640, 640)
[array([[[5.23447227e+00, 6.69266605e+00, 8.65236187e+00, ...,
         4.79491383e-01, 3.37575376e-02, 3.31561863e-02],
        [8.68288898e+00, 6.10372543e+00, 1.29462328e+01, ...,
         6.08258843e-01, 3.95277739e-02, 2.54235268e-02],
        [1.86500130e+01, 4.81172848e+00, 2.00691833e+01, ...,
         4.51061219e-01, 3.55225503e-02, 3.66864204e-02],
        ...,
        [5.58287659e+02, 6.22580444e+02, 4.02449890e+02, ...,
         1.54093772e-01, 1.39918208e-01, 1.17276877e-01],
        [5.91142090e+02, 6.22777344e+02, 3.80226379e+02, ...,
         1.54447049e-01, 1.27226621e-01, 1.10112160e-01],
        [6.23208191e+02, 6.23160950e+02, 4.20902435e+02, ...,
         1.52335852e-01, 1.48618966e-01, 1.11448884e-01]]], dtype=float32)]


In [32]:
import numpy as np

def extract_class_labels(outputs):
    # Convert the list to a NumPy array
    array = np.array(outputs)
    
    # Get the number of boxes
    num_boxes = array.shape[1] // 6  # Assuming 6 elements per box (x, y, w, h, conf, cls)
    
    # Extract class IDs
    class_ids = array[:, 5]  # Column 5 contains class IDs
    
    return class_ids

# Assuming ort_outs is your list of outputs
class_ids = extract_class_labels(ort_outs)

print(class_ids)

IndexError: index 5 is out of bounds for axis 1 with size 1

In [34]:
np.shape(ort_outs)

(1, 1, 25200, 10)

In [37]:
def extract_class_labels(array):
    # Convert to numpy array if it's not already
    array = np.array(array)
    
    # Reshape the array if needed
    if len(array.shape) == 4:
        array = array.reshape(-1, array.shape[-1])
    
    print(f"Processed array shape: {array.shape}")
    
    # Check if the array has at least 10 columns
    if array.shape[1] >= 10:
        # Extract class IDs (assuming they are in the last column, index 9)
        class_ids = array[:, 9].astype(int)
        
        # Extract probabilities (assuming the first 9 columns contain probabilities)
        probabilities = array[:, :9]
        
        # Find the index of the highest probability for each detection
        max_prob_indices = np.argmax(probabilities, axis=1)
        max_prob_values = np.max(probabilities, axis=1)
        
        # Return class IDs and probabilities
        return class_ids, max_prob_indices, max_prob_values
    else:
        print("Unexpected array shape. Cannot extract class IDs and probabilities.")
        return [], [], []
class_ids, max_prob_indices, max_prob_values = extract_class_labels(ort_outs)



Processed array shape: (25200, 10)


In [42]:
class_ids
max_prob_indices
len(class_ids)

25200

In [9]:
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
def load_model(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    return session

# Preprocess the image for YOLOv5 (resize, normalize, and convert to tensor format)
def preprocess_image(image_path, input_size=(640, 640)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0  # Normalize to [0, 1]
    img_transposed = np.transpose(img_normalized, (2, 0, 1))  # HWC to CHW format
    img_tensor = np.expand_dims(img_transposed, axis=0).astype(np.float32)  # Add batch dimension
    return img_tensor, img

# Post-process the model output to extract bounding boxes, class labels, and probabilities
def postprocess_output(output, img_shape, conf_threshold=0.5, iou_threshold=0.4):
    boxes, scores, class_ids = [], [], []
    detections = output[0]
    img_h, img_w = img_shape[:2]

    # Parse each detection (x, y, w, h, conf, class_score1, class_score2, ..., class_scoreN)
    for detection in detections:
        scores = detection[4:]  # All the class scores
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            # Box is (center_x, center_y, width, height) - convert to x1, y1, x2, y2
            box = detection[:4] * np.array([img_w, img_h, img_w, img_h])
            x_center, y_center, width, height = box
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            boxes.append([x1, y1, x2, y2])
            scores.append(confidence)
            class_ids.append(class_id)
    
    return boxes, scores, class_ids

# Map class IDs to class names
class_names = ['pf', 'pv', 'po', 'pm', 'wbc']

# Inference function
def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Print output shape for debugging
    print("Model output shape:", outputs[0].shape)

    # You can comment out the postprocess for now until we verify the output format
    # boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    return outputs

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_path = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/PO/po_52.jpg'  # Update with your image path

num_parasites, detection_results = run_inference(onnx_model_path, image_path)

print(f"Number of detected parasites: {num_parasites}")
for label, confidence in detection_results:
    print(f"Detected {label} with confidence {confidence:.2f}")


Model output shape: (1, 25200, 10)


ValueError: not enough values to unpack (expected 2, got 1)

In [12]:
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
def load_model(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    return session

# Preprocess the image for YOLOv5 (resize, normalize, and convert to tensor format)
def preprocess_image(image_path, input_size=(640, 640)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0  # Normalize to [0, 1]
    img_transposed = np.transpose(img_normalized, (2, 0, 1))  # HWC to CHW format
    img_tensor = np.expand_dims(img_transposed, axis=0).astype(np.float32)  # Add batch dimension
    return img_tensor, img

# Post-process the model output to extract bounding boxes, class labels, and probabilities
def postprocess_output(output, img_shape, conf_threshold=0.5):
    # Unpack the first output (the detections)
    detections = output[0][0]  # Shape is (25200, 10)
    
    img_h, img_w = img_shape[:2]
    
    boxes, confidences, class_ids = [], [], []

    # Loop through all the detections
    for detection in detections:
        # Extract bounding box coordinates and objectness score
        x_center, y_center, width, height, objectness = detection[:5]

        # Extract class scores (the last 5 values)
        class_scores = detection[5:]

        # Get the class with the highest score
        class_id = np.argmax(class_scores)
        class_confidence = class_scores[class_id]

        # Multiply objectness score with class confidence to get the final confidence score
        confidence = objectness * class_confidence

        # Filter out weak detections based on confidence threshold
        if confidence > conf_threshold:
            # Convert from (center_x, center_y, width, height) to (x1, y1, x2, y2)
            x1 = int((x_center - width / 2) * img_w)
            y1 = int((y_center - height / 2) * img_h)
            x2 = int((x_center + width / 2) * img_w)
            y2 = int((y_center + height / 2) * img_h)
            
            boxes.append([x1, y1, x2, y2])
            confidences.append(confidence)
            class_ids.append(class_id)
    
    return boxes, confidences, class_ids

# Map class IDs to class names
class_names = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

# Inference function
def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Post-process the output
    boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    # Number of detected parasites
    num_parasites = len(class_ids)
    
    # List each detected parasite with label and confidence
    results = []
    for i in range(num_parasites):
        label = class_names[class_ids[i]]
        confidence = confidences[i]
        results.append((label, confidence))

    return num_parasites, results

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_path = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/Processed/pv_105.jpg'  # Update with your image path

num_parasites, detection_results = run_inference(onnx_model_path, image_path)

print(f"Number of detected parasites: {num_parasites}")
for label, confidence in detection_results:
    print(f"Detected {label} with confidence {confidence:.2f}")


Number of detected parasites: 22
Detected po with confidence 0.72
Detected po with confidence 0.73
Detected pv with confidence 0.51
Detected pv with confidence 0.61
Detected po with confidence 0.68
Detected po with confidence 0.69
Detected pv with confidence 0.56
Detected po with confidence 0.69
Detected po with confidence 0.71
Detected pv with confidence 0.69
Detected pv with confidence 0.60
Detected po with confidence 0.76
Detected po with confidence 0.78
Detected pv with confidence 0.70
Detected pv with confidence 0.57
Detected po with confidence 0.74
Detected po with confidence 0.76
Detected pv with confidence 0.69
Detected pv with confidence 0.59
Detected pv with confidence 0.65
Detected pv with confidence 0.61
Detected pv with confidence 0.67


In [25]:
from collections import defaultdict
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
def load_model(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    return session

# Preprocess the image for YOLOv5 (resize, normalize, and convert to tensor format)
def preprocess_image(image_path, input_size=(640, 640)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0  # Normalize to [0, 1]
    img_transposed = np.transpose(img_normalized, (2, 0, 1))  # HWC to CHW format
    img_tensor = np.expand_dims(img_transposed, axis=0).astype(np.float32)  # Add batch dimension
    return img_tensor, img

# Post-process the model output to extract bounding boxes, class labels, and probabilities
def postprocess_output(output, img_shape, conf_threshold=0.5):
    detections = output[0][0]  # Shape is (25200, 10)
    
    img_h, img_w = img_shape[:2]
    
    boxes, confidences, class_ids = [], [], []

    for detection in detections:
        # Extract bounding box coordinates and objectness score
        x_center, y_center, width, height, objectness = detection[:5]
        class_scores = detection[5:]
        class_id = np.argmax(class_scores)
        class_confidence = class_scores[class_id]
        confidence = objectness * class_confidence

        if confidence > conf_threshold:
            x1 = int((x_center - width / 2) * img_w)
            y1 = int((y_center - height / 2) * img_h)
            x2 = int((x_center + width / 2) * img_w)
            y2 = int((y_center + height / 2) * img_h)
            
            boxes.append([x1, y1, x2, y2])
            confidences.append(confidence)
            class_ids.append(class_id)
    
    return boxes, confidences, class_ids

# Map class IDs to class names
class_names = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

# Function to calculate average confidence for each parasite type and find the dominant one
def get_dominant_parasite(class_ids, confidences):
    # Dictionary to hold confidence lists for each class
    class_confidence_map = defaultdict(list)
    
    # Group confidences by class
    for class_id, confidence in zip(class_ids, confidences):
        class_confidence_map[class_id].append(confidence)

    # Calculate average confidence for each class and find the dominant one
    average_confidences = {class_id: np.mean(conf_list) for class_id, conf_list in class_confidence_map.items()}
    

    print(average_confidences)
    # Get the class with the highest average confidence
    dominant_class_id = max(average_confidences, key=average_confidences.get)
    
    # Return the class name and its average confidence
    return class_names[dominant_class_id], average_confidences[dominant_class_id]

# Inference function
def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Post-process the output
    boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    # If parasites are detected, calculate dominant parasite type
    if len(class_ids) > 0:
        dominant_parasite, avg_confidence = get_dominant_parasite(class_ids, confidences)
        return len(class_ids), dominant_parasite, avg_confidence
    else:
        return 0, None, None  # No parasites detected

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_path = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/Raw/pm_62.jpg'  # Update with your image path

num_parasites, dominant_parasite, avg_confidence = run_inference(onnx_model_path, image_path)

if num_parasites > 0:
    print(f"Number of detected parasites: {num_parasites}")
    print(f"Dominant parasite type: {dominant_parasite} with average confidence {avg_confidence:.2f}")
else:
    print("No parasites detected")


{1: 0.6247605, 4: 0.7140656}
Number of detected parasites: 65
Dominant parasite type: wbc with average confidence 0.71


In [17]:
import os
from collections import defaultdict

# Function to calculate parasite density using WHO method
def calculate_parasite_density(parasite_count, wbc_count, wbc_standard=8000):
    if wbc_count == 0:
        return 0  # Avoid division by zero
    return (parasite_count / wbc_count) * wbc_standard

# Function to classify severity based on parasite density
def classify_severity(parasite_density):
    if parasite_density < 1000:
        return "Mild"
    elif 1000 <= parasite_density <= 10000:
        return "Moderate"
    else:
        return "Severe"
    
# Map class IDs to class names
class_ids = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

# Process multiple images
def process_images(onnx_model_path, image_dir):
    # Initialize total counts
    total_parasites = 0
    total_wbcs = 0
    image_results = []

    # Loop over all images in the directory
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        print(f"Processing {image_file}...")

        # Run inference on the image
        num_parasites, dominant_parasite, avg_confidence = run_inference(onnx_model_path, image_path)

        # If parasites are detected, add to total count
        if num_parasites > 0:
            # Count WBCs separately (assuming class ID 4 is WBC)
            wbc_count = sum(1 for class_id in class_ids if class_id == 4)
            parasite_count = num_parasites - wbc_count  # Exclude WBCs from parasite count
            
            # Add to total counts
            total_parasites += parasite_count
            total_wbcs += wbc_count

            # Store result for this image
            image_results.append({
                "image": image_file,
                "parasite_count": parasite_count,
                "wbc_count": wbc_count
            })
    
    # Calculate the overall parasite density using WHO formula
    parasite_density = calculate_parasite_density(total_parasites, total_wbcs)
    severity = classify_severity(parasite_density)

    # Return the results
    return {
        "total_parasites": total_parasites,
        "total_wbcs": total_wbcs,
        "parasite_density": parasite_density,
        "severity": severity,
        "image_results": image_results
    }

# Directory where images are stored
image_dir = '/path_to_image_directory/'  # Update with your directory containing images
onnx_model_path = '/path_to_your_model.onnx'  # Update with your ONNX model path

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_dir = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/External'  # Update with your image path

# Process images and calculate parasite density
results = process_images(onnx_model_path, image_dir)

# Print results
print(f"Total Parasites Detected: {results['total_parasites']}")
print(f"Total WBCs Detected: {results['total_wbcs']}")
print(f"Parasite Density (per µL): {results['parasite_density']:.2f}")
print(f"Malaria Severity: {results['severity']}")

# If you want to display individual image results
for image_result in results["image_results"]:
    print(f"Image {image_result['image']}: Parasites: {image_result['parasite_count']}, WBCs: {image_result['wbc_count']}")


Processing pf_6.jpg...
{0: 0.5948736, 4: 0.72652686}
Processing pf_7.jpg...
{0: 0.5997648, 4: 0.70810425}
Processing pf_5.jpg...
{0: 0.5748825}
Processing pf_4.jpg...
{4: 0.75644255, 0: 0.60003173}
Processing pf_1.jpg...
{0: 0.54353255, 4: 0.7458056}
Processing pf_3.jpg...
{0: 0.59092695, 2: 0.5121875, 4: 0.62406975}
Processing pf_2.jpg...
{0: 0.5892726, 4: 0.7061373}
Processing pf_8.jpg...
{0: 0.5775399}
Total Parasites Detected: 797
Total WBCs Detected: 8
Parasite Density (per µL): 797000.00
Malaria Severity: Severe
Image pf_6.jpg: Parasites: 104, WBCs: 1
Image pf_7.jpg: Parasites: 127, WBCs: 1
Image pf_5.jpg: Parasites: 104, WBCs: 1
Image pf_4.jpg: Parasites: 106, WBCs: 1
Image pf_1.jpg: Parasites: 71, WBCs: 1
Image pf_3.jpg: Parasites: 113, WBCs: 1
Image pf_2.jpg: Parasites: 92, WBCs: 1
Image pf_8.jpg: Parasites: 80, WBCs: 1


In [24]:
import os
from collections import defaultdict
def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Post-process the output to get bounding boxes, confidences, and class IDs
    boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    # Return results
    return len(class_ids), None, None, confidences, class_ids
# Function to calculate parasite density using WHO method
def calculate_parasite_density(parasite_count, wbc_count, wbc_standard=8000):
    if wbc_count == 0:
        return 0  # Avoid division by zero
    return (parasite_count / wbc_count) * wbc_standard

# Function to classify severity based on parasite density
def classify_severity(parasite_density):
    if parasite_density < 1000:
        return "Mild"
    elif 1000 <= parasite_density <= 10000:
        return "Moderate"
    else:
        return "Severe"
    
# Map class IDs to class names
class_names = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

def process_images(onnx_model_path, image_dir):
    # Initialize total counts
    total_parasites = 0
    total_wbcs = 0
    parasite_confidences = defaultdict(list)  # To store confidences of each parasite type
    image_results = []

    # Loop over all images in the directory
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        print(f"Processing {image_file}...")

        # Run inference on the image
        num_parasites, dominant_parasite, avg_confidence, confidences, class_ids = run_inference(onnx_model_path, image_path)

        # If parasites are detected, add to total count
        if num_parasites > 0:
            # Count WBCs separately (assuming class ID 4 is WBC)
            wbc_count = sum(1 for class_id in class_ids if class_id == 4)
            parasite_count = num_parasites - wbc_count  # Exclude WBCs from parasite count
            
            # Add to total counts
            total_parasites += parasite_count
            total_wbcs += wbc_count

            # Track the confidences for each parasite type
            for class_id, confidence in zip(class_ids, confidences):
                if class_id != 4:  # Skip WBCs
                    parasite_confidences[class_id].append(confidence)

            # Store result for this image
            image_results.append({
                "image": image_file,
                "parasite_count": parasite_count,
                "wbc_count": wbc_count
            })
    
    # Calculate the overall parasite density using WHO formula
    parasite_density = calculate_parasite_density(total_parasites, total_wbcs)
    severity = classify_severity(parasite_density)

    # Determine the dominant parasite type based on average confidence
    average_confidences = {class_id: sum(conf_list) / len(conf_list) for class_id, conf_list in parasite_confidences.items()}
    dominant_parasite_id = max(average_confidences, key=average_confidences.get)
    dominant_parasite = class_names[dominant_parasite_id]
    dominant_parasite_avg_confidence = average_confidences[dominant_parasite_id]

    # Return the results
    return {
        "total_parasites": total_parasites,
        "total_wbcs": total_wbcs,
        "parasite_density": parasite_density,
        "severity": severity,
        "dominant_parasite": dominant_parasite,
        "dominant_parasite_avg_confidence": dominant_parasite_avg_confidence,
        "image_results": image_results
    }

    # Calculate the overall parasite density using WHO formula
    parasite_density = calculate_parasite_density(total_parasites, total_wbcs)
    severity = classify_severity(parasite_density)

    # Calculate the average confidence for each parasite type
    average_confidences = {class_id: sum(conf_list) / len(conf_list) 
                           for class_id, conf_list in parasite_confidences.items()}

    # Determine the dominant parasite type based on the highest average confidence
    if average_confidences:
        dominant_class_id = max(average_confidences, key=average_confidences.get)
        dominant_parasite = class_names[dominant_class_id]
        dominant_confidence = average_confidences[dominant_class_id]
    else:
        dominant_parasite, dominant_confidence = None, None

    # Return the results
    return {
        "total_parasites": total_parasites,
        "total_wbcs": total_wbcs,
        "parasite_density": parasite_density,
        "severity": severity,
        "dominant_parasite": dominant_parasite,
        "dominant_confidence": dominant_confidence,
        "image_results": image_results
    }

# Directory where images are stored
image_dir = '/path_to_image_directory/'  # Update with your directory containing images
onnx_model_path = '/path_to_your_model.onnx'  # Update with your ONNX model path

# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_dir = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/Raw'  # Update with your image path

# Process images and calculate parasite density
results = process_images(onnx_model_path, image_dir)

# Print results
print(f"Total Parasites Detected: {results['total_parasites']}")
print(f"Total WBCs Detected: {results['total_wbcs']}")
print(f"Parasite Density (per µL): {results['parasite_density']:.2f}")
print(f"Malaria Severity: {results['severity']}")

# Print dominant parasite type and its average confidence
if results['dominant_parasite']:
    print(f"Dominant Parasite Type: {results['dominant_parasite']} with average confidence {results['dominant_parasite_avg_confidence']:.2f}")
else:
    print("No dominant parasite detected")

# If you want to display individual image results
for image_result in results["image_results"]:
    print(f"Image {image_result['image']}: Parasites: {image_result['parasite_count']}, WBCs: {image_result['wbc_count']}")


Processing pm_64.jpg...
Processing pm_65.jpg...
Processing pm_67.jpg...
Processing pm_66.jpg...
Processing pm_62.jpg...
Processing pm_63.jpg...
Total Parasites Detected: 58
Total WBCs Detected: 283
Parasite Density (per µL): 1639.58
Malaria Severity: Moderate
Dominant Parasite Type: pm with average confidence 0.63
Image pm_64.jpg: Parasites: 0, WBCs: 62
Image pm_65.jpg: Parasites: 5, WBCs: 24
Image pm_67.jpg: Parasites: 12, WBCs: 35
Image pm_66.jpg: Parasites: 1, WBCs: 20
Image pm_62.jpg: Parasites: 37, WBCs: 28
Image pm_63.jpg: Parasites: 3, WBCs: 114


In [33]:
import os
from collections import defaultdict

# Function to calculate parasite density using WHO method
def calculate_parasite_density(parasite_count, wbc_count, wbc_standard=8000):
    if wbc_count == 0:
        return 0  # Avoid division by zero
    return (parasite_count / wbc_count) * wbc_standard
def postprocess_output(output, img_shape, parasite_threshold=0.75, wbc_threshold=0.6):
    detections = output[0][0]  # Shape is (25200, 10)
    
    img_h, img_w = img_shape[:2]
    
    boxes, confidences, class_ids = [], [], []

    # Loop through all detections
    for detection in detections:
        # Extract bounding box coordinates and objectness score
        x_center, y_center, width, height, objectness = detection[:5]
        class_scores = detection[5:]
        class_id = np.argmax(class_scores)
        class_confidence = class_scores[class_id]
        confidence = objectness * class_confidence

        # Apply different thresholds based on the detected class (parasite vs WBC)
        if (class_id in [0, 1, 2, 3] and confidence > parasite_threshold) or (class_id == 4 and confidence > wbc_threshold):
            # Convert from (center_x, center_y, width, height) to (x1, y1, x2, y2)
            x1 = int((x_center - width / 2) * img_w)
            y1 = int((y_center - height / 2) * img_h)
            x2 = int((x_center + width / 2) * img_w)
            y2 = int((y_center + height / 2) * img_h)
            
            boxes.append([x1, y1, x2, y2])
            confidences.append(confidence)
            class_ids.append(class_id)
    
    return boxes, confidences, class_ids

# Function to classify severity based on parasite density
def classify_severity(parasite_density):
    if parasite_density < 1000:
        return "Mild"
    elif 1000 <= parasite_density <= 10000:
        return "Moderate"
    else:
        return "Severe"

# Map class IDs to class names
class_names = {0: 'pf', 1: 'pm', 2: 'po', 3: 'pv', 4: 'wbc'}

def run_inference(onnx_model_path, image_path):
    # Load model
    session = load_model(onnx_model_path)

    # Preprocess the image
    input_tensor, original_img = preprocess_image(image_path)

    # Run the model inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    # Post-process the output to get bounding boxes, confidences, and class IDs
    boxes, confidences, class_ids = postprocess_output(outputs, original_img.shape)

    # Return results
    return len(class_ids), confidences, class_ids

def process_images(onnx_model_path, image_dir):
    # Initialize total counts
    total_parasites = 0
    total_wbcs = 0
    parasite_confidences = defaultdict(list)  # To store confidences of each parasite type
    image_results = []

    # Loop over all images in the directory
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        print(f"Processing {image_file}...")

        # Run inference on the image
        num_parasites, confidences, class_ids = run_inference(onnx_model_path, image_path)

        # If parasites are detected, add to total count
        if num_parasites > 0:
            # Count WBCs separately (assuming class ID 4 is WBC)
            wbc_count = sum(1 for class_id in class_ids if class_id == 4)
            parasite_count = num_parasites - wbc_count  # Exclude WBCs from parasite count
            
            # Add to total counts
            total_parasites += parasite_count
            total_wbcs += wbc_count

            # Track the confidences for each parasite type
            for class_id, confidence in zip(class_ids, confidences):
                if class_id != 4:  # Skip WBCs
                    parasite_confidences[class_id].append(confidence)

            # Store result for this image
            image_results.append({
                "image": image_file,
                "parasite_count": parasite_count,
                "wbc_count": wbc_count
            })
    
    # Calculate the overall parasite density using WHO formula
    parasite_density = calculate_parasite_density(total_parasites, total_wbcs)
    severity = classify_severity(parasite_density)

    # Calculate the average confidence for each parasite type
    average_confidences = {class_id: sum(conf_list) / len(conf_list) 
                           for class_id, conf_list in parasite_confidences.items()}

    # Determine the dominant parasite type based on the highest average confidence
    if average_confidences:
        dominant_class_id = max(average_confidences, key=average_confidences.get)
        dominant_parasite = class_names[dominant_class_id]
        dominant_confidence = average_confidences[dominant_class_id]
    else:
        dominant_parasite, dominant_confidence = None, None

    # Return the results
    return {
        "total_parasites": total_parasites,
        "total_wbcs": total_wbcs,
        "parasite_density": parasite_density,
        "severity": severity,
        "dominant_parasite": dominant_parasite,
        "dominant_confidence": dominant_confidence,
        "image_results": image_results
    }

# Directory where images are stored
# Test the function
onnx_model_path = '/Users/alaindestinkarasira/Documents/MALARIA/Malaria_Pjct/MalariaDiagnosis/app/models/new_best.onnx'  # Update with your ONNX model path
image_dir = '/Users/alaindestinkarasira/Documents/MALARIA/ImageData/External'  # Update with your image path

# Process images and calculate parasite density
results = process_images(onnx_model_path, image_dir)

# Print results
print(f"Total Parasites Detected: {results['total_parasites']}")
print(f"Total WBCs Detected: {results['total_wbcs']}")
print(f"Parasite Density (per µL): {results['parasite_density']:.2f}")
print(f"Malaria Severity: {results['severity']}")

# Print dominant parasite type and its average confidence
if results['dominant_parasite']:
    print(f"Dominant Parasite Type: {results['dominant_parasite']} with average confidence {results['dominant_confidence']:.2f}")
else:
    print("No dominant parasite detected")

# If you want to display individual image results
for image_result in results["image_results"]:
    print(f"Image {image_result['image']}: Parasites: {image_result['parasite_count']}, WBCs: {image_result['wbc_count']}")


Processing pf_6.jpg...
Processing pf_7.jpg...
Processing pf_5.jpg...
Processing pf_4.jpg...
Processing pf_1.jpg...
Processing pf_3.jpg...
Processing pf_2.jpg...
Processing pf_8.jpg...
Total Parasites Detected: 11
Total WBCs Detected: 143
Parasite Density (per µL): 615.38
Malaria Severity: Mild
Dominant Parasite Type: pf with average confidence 0.76
Image pf_6.jpg: Parasites: 3, WBCs: 12
Image pf_7.jpg: Parasites: 3, WBCs: 44
Image pf_4.jpg: Parasites: 4, WBCs: 16
Image pf_1.jpg: Parasites: 0, WBCs: 27
Image pf_3.jpg: Parasites: 0, WBCs: 16
Image pf_2.jpg: Parasites: 1, WBCs: 28
